# Basic Statistics

G. Richards
(2016, 2018, 2020)

Resources for this material include Ivezic Sections 1.2, 3.0, and 3.1, Karen' Leighly's [Bayesian Statistics Lecture](http://seminar.ouml.org/lectures/bayesian-statistics/), and [Jo Bovy's 2016 class](http://astro.utoronto.ca/%7Ebovy/teaching.html), specifically Lecture 1.

Last time we worked through some examples of the kinds of things that we will be doing later in the course.  But before we can do the fun stuff, we need to lay some statistical groundwork.  Some of you may have encountered some of this material in Math 311.

## Notation

First we need to go over some of the notation that the book uses.   

$x$ is a scalar quantity, measured $N$ times

$x_i$ is a single measurement with $i=1,...,N$

$\{x_i\}$ refers to the set of all N measurements

We are generally trying to *estimate* $h(x)$, the ***true*** distribution from which the values of $x$ are drawn. We will refer to $h(x)$ as the probability density (distribution) function or the "**pdf**" and $h(x)dx$ is the propobability of a value lying between $x$ and $x+dx$. A histogram is an example of a pdf.

While $h(x)$ is the "true" distribution (or **population** pdf), what we *measure* from the data is the ***empirical*** distribution, which is denoted $f(x)$.  So, $f(x)$ is a *model* of $h(x)$.  In principle, with infinite data $f(x) \rightarrow h(x)$, but in reality measurement errors keep this from being strictly true.

If we are attempting to guess a *model* for $h(x)$, then the process is *parametric*.  With a model solution we can generate new (simulated) data that should mimic what we measure.  

If we are not attempting to guess a model, then the process is *nonparametic*.  That is we are just trying to describe the data that we see in the most compact manner that we can, but we are not trying to produce mock data.

The histograms that we made last time are an example of a nonparametric method of describing data.

## Goal

We could summarize the goal of the first few weeks of this class as an attempt to: 
1. estimate $f(x)$ from some real (possibly multi-dimensional) data set, 
2. find a way to describe $f(x)$ and its uncertainty, 
3. compare it to models of $h(x)$, and then 
4. use the knowledge that we have gained in order to interpret new measurements.

## Probability

The probability of $A$, $p(A)$, is the probability that some event will happen (say a coin toss coming up tails), or if the process is continuous, the probability of $A$ falling in a certain range.  (N.B., Technically these two things are different and sometimes are indicated by $P$ and $p$, but I'm ignoring that here.)  $p(A)$ must be positive definite for all $A$ and the sum/integral of the pdf must be unity.

If we have two events, $A$ and $B$, the possible combinations are illustrated by the following figure:
![Figure 3.1](http://www.astroml.org/_images/fig_prob_sum_1.png)

$A \cup B$ is the *union* of sets $A$ and $B$.

$A \cap B$ is the *intersection* of sets $A$ and $B$.

The probability that *either* $A$ or $B$ will happen (which could include both) is the *union*, given by

$$p(A \cup B) = p(A) + p(B) - p(A \cap B)$$

The figure makes it clear why the last term is necessary.  Since $A$ and $B$ overlap, we are double-counting the region where *both* $A$ and $B$ happen, so we have to subtract this out.  


The probability that *both* $A$ and $B$ will happen, $p(A \cap B)$, can be written as
$$p(A \cap B) = p(A|B)p(B) = p(B|A)p(A)$$

where p(A|B) is the probability of A *given that* B is true and is called the *conditional probability*.  So the $|$ is short for "given that".

The *law of total probability* says that

$$p(A) = \sum_ip(A|B_i)p(B_i)$$

Example:

    A = hit head on door frame, B = { is tall, is average, is short }
    P(A) = P(A|is tall) + P(A|is average) + P(A|is short)

N.B.  Just to be annoying, different people use different notation and the following all mean the same thing
$$p(A \cap B) = p(A,B) = p(AB) = p(A \,{\rm and}\, B)$$

I'll use the comma notation as that is what the book uses.  So, read the comma like "and".

It is important to realize that the following is *always* true
$$p(A,B) = p(A|B)p(B) = p(B|A)p(A)$$

However, if $A$ and $B$ are independent, then 

$$p(A,B) = p(A)p(B)$$

Example:

     Vera is successful and Vera is a Libra.
     
In other words, knowing A happened tells us nothing about whether B happened (or will happen), and vice versa.

Let's look an example.

If you have a bag with 5 marbles (3 yellow and 2 blue), and you want to know the probability of picking 2 yellow marbles in a row, that would be

$$p(Y_1,Y_2) = p(Y_2|Y_1)p(Y_1).$$

$p(Y_1) = \frac{3}{5}$ since you have an equally likely chance of drawing any of the 5 marbles.

If you did not put the first marble back in the back after drawing it (sampling *without* "replacement"), then the probability
$p(Y_2|Y_1) = \frac{2}{4}$, so that
$$p(Y_1,Y_2) = \frac{3}{5}\frac{2}{4} = \frac{3}{10}.$$

But if you put the first marble back, then
$p(Y_2|Y_1) = \frac{3}{5} = p(Y_2)$, so that 
$$p(Y_1,Y_2) = \frac{3}{5}\frac{3}{5} = \frac{9}{25}.$$

In the first case $A$ and $B$ (or rather $Y_1$ and $Y_2$) are *not* independent, whereas in the second case they are.

We say that two random variables, $A$ and $B$ are independent *iff*
$p(A,B) = p(A)p(B)$
such that knowing $B$ does not give any information about $A$.

A more complicated example from Jo Bovy's class at UToronto
![Bovy_L1-StatMiniCourse_page21](figures/Bovy_L1-StatMiniCourse_page21.png)

So
$$p(A \,{\rm or}\, B|C) = p(A|C) + p(B|C) - p(A \, {\rm and}\, B|C)$$

We could get even more complicated than that, but let's leave it there for now as this is all that we need right now.

Need more help with this?  Try watching some Khan Academy videos and working through the exercises:
[https://www.khanacademy.org/math/probability/probability-geometry](https://www.khanacademy.org/math/probability/probability-geometry)

[https://www.khanacademy.org/math/precalculus/prob-comb](https://www.khanacademy.org/math/precalculus/prob-comb)

## Bayes' Rule

We have that 
$$p(x,y) = p(x|y)p(y) = p(y|x)p(x)$$

We can define the ***marginal probability*** as
$$p(x) = \int p(x,y)dy,$$
where marginal means essentially projecting on to one axis (integrating over the other axis).

We can re-write this as
$$p(x) = \int p(x|y)p(y) dy$$

This is just the law of total probability (as defined above), but for continous variables.

An illustration might help.  In the following figure, we have a 2-D distribution in $x-y$ parameter space.  Here $x$ and $y$ are *not* independent as, once you pick a $y$, your values of $x$ are constrained.

The *marginal* distributions are shown on the left and bottom sides of the left panel.  As the equation above says, this is just the integral along the $x$ direction for a given $y$ (left side panel) or the integral along the $y$ direction for a given $x$ (bottom panel).  

The three panels on the right show the *conditional* probability (of $x$) for three $y$ values: $p(x|y=y_0)$.  These are just "slices" through the 2-D distribution.

![http://www.astroml.org/_images/fig_conditional_probability_1.png](http://www.astroml.org/_images/fig_conditional_probability_1.png)


Since $p(x|y)p(y) = p(y|x)p(x)$ we can write that
$$p(y|x) = \frac{p(x|y)p(y)}{p(x)} = \frac{p(x|y)p(y)}{\int p(x|y)p(y) dy}$$
which in words says that

> the (conditional) probability of $y$ given $x$ is just the (conditional) probability of $x$ given $y$ times the (marginal) probability of $y$ divided by the (marginal) probability of $x$, where the latter is just the integral of the numerator.

This is **Bayes' rule**, which itself is not at all controverial, though its application can be as we'll discuss later.

## Example: Lego's 

An example with Lego's (it's awesome):
[https://www.countbayesie.com/blog/2015/2/18/bayes-theorem-with-lego](https://www.countbayesie.com/blog/2015/2/18/bayes-theorem-with-lego)

## Example: Monty Hall Problem

You are playing a game show and are shown 2 doors.  One has a car behind it, the other a goat.  What are your chances of picking the door with the car?

OK, now there are 3 doors: one with a car, two with goats.  The game show host asks you to pick a door, but not to open it yet.  Then the host opens one of the other two doors (that you did not pick), making sure to select one with a goat.  The host offers you the opportunity to switch doors.  Do you?

![https://upload.wikimedia.org/wikipedia/commons/thumb/3/3f/Monty_open_door.svg/180px-Monty_open_door.svg.png](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3f/Monty_open_door.svg/180px-Monty_open_door.svg.png)

Now you are back at the 2 door situation.  But what can you make of your prior information?

$p(1{\rm st \; choice}) = 1/3$

$p({\rm other}) = 2/3$
which doesn't change after host opens door without the prize.
So, switching doubles your chances.  But only because you had prior information.  If someone walked in after the "bad" door was opened, then their probability of winning is the expected $1/2$.

Try it:
https://betterexplained.com/articles/understanding-the-monty-hall-problem/

This example is easier to understand if you do the same thing, but start with a much larger number of doors.

For $N$ choices, revealing $N-2$ "answers" doesn't change the probability of your choice.  It is still $\frac{1}{N}$.  But it *does* change the probability of your knowledge of the *other* remaining choice by $N-1$ and it is $\frac{N-1}{N}$.

This is an example of the use of *conditional* probability, where we have $p(A|B) \ne p(A)$.



## Example: Contingency Table

We can also use Bayes' rule to learn something about false positives and false negatives.

Let's say that we have a test for a disease.  The test can be positive ($T=1$) or negative ($T=0$) and one can either have the disease ($D=1$) or not ($D=0$).  So, there are 4 possible combinations:
$$T=0; D=0 \;\;\;  {\rm true \; negative}$$
$$T=0; D=1 \;\;\; {\rm false \; negative}$$
$$T=1; D=0 \;\;\; {\rm false \; positive}$$
$$T=1; D=1 \;\;\; {\rm true \; positive}$$

All else being equal, you have a 50% chance of being misdiagnosed.  Not good!  But the probability of disease and the accuracy of the test presumably are not random.

If the rates of false positive and false negative are:
$$p(T=1|D=0) = \epsilon_{\rm FP}$$
$$p(T=0|D=1) = \epsilon_{\rm FN}$$

then the true negative and true positive rates are just:
$$p(T=0| D=0) = 1-\epsilon_{\rm FP}$$
$$p(T=1| D=1) = 1-\epsilon_{\rm FN}$$

Let's assume that $\epsilon_{\rm FP}=0.02$ and $\epsilon_{\rm FN}=0.001$. 

In graphical form this is:
![http://www.astroml.org/_images/fig_contingency_table_1.png](http://www.astroml.org/_images/fig_contingency_table_1.png)

If we have a **prior** regarding how likely the disease is, we can take this into account.

$$p(D=1)=\epsilon_D$$

and then $p(D=0)=1-\epsilon_D$. Say, $\epsilon_D = 0.01$. 

Now assume that a person tested positive. What is the probability that this person has the disease?

We can't just read $p(D=1|T=1)$ off the table.  That's because the table entry is the conditional probability of the *test* given the *data*, $p(T=1|D=1)$, what we want is the conditional probability of the *data* given the *test*.

Bayes' rule then can be used to help us determine how likely it is that you have the disease if you tested positive:

$$p(D=1|T=1) = \frac{p(T=1|D=1)p(D=1)}{p(T=1)},$$

where $$p(T=1) = p(T=1|D=0)p(D=0) + p(T=1|D=1)p(D=1).$$

So
$$p(D=1|T=1) = \frac{(1 - \epsilon_{FN})\epsilon_D}{\epsilon_{FP}(1-\epsilon_D) + (1-\epsilon_{FN})\epsilon_D} \approx \frac{\epsilon_D}{\epsilon_D+\epsilon_{FP}}$$

That means that to get a reliable diagnosis, we need $\epsilon_{FP}$ to be quite small.  (Because you *want* the probability to be close to unity if you test positive, otherwise it is a *false* positive).

In our example with a disease rate of 1% ($\epsilon_D = 0.01$) and a false positive rate of 2% ($\epsilon_{\rm FP}=0.02$), we have 

$$p(D=1|T=1) = \frac{0.01}{0.01+0.02} = 0.333$$

Then in a sample of 1000 people, 10 people will *actually* have the disease $(1000*0.01)$, but another 20 $(1000*0.02)$ will test positive, despite being healthy!

Therefore, in that sample of 30 people who tested positive, only 1/3 have the disease (not 98% or 99.9% as you might have expected!).

Here's a [summary/example](https://towardsdatascience.com/false-positives-negatives-and-bayes-rule-for-covid-19-testing-750eaba84acd) from the perspective of the 2020 global pandemic.

## Models and Data 

In this class, we generally won't be dealing with the probability of events $A$ and $B$, rather we will be dealing with models and data, where we are trying to determine the model, given the data.  So, we can rewrite Bayes' rule as
$$p({\rm model}|{\rm data}) = \frac{p(\rm{data}|\rm{model})p(\rm{model})}{p(\rm{data})}.$$

We can write this in words as:
$${\rm Posterior Probability} = \frac{{\rm Likelihood}\times{\rm Prior}}{{\rm Evidence}},$$

where we interpret the posterior probability as the probability of the model (including the model parameters).

We'll talk more about models next time.

----

## Distributions

Our goal is ultimately to figure out the *distribution* from which our data is drawn, i.e., we want to know the *model*.  For example, let's say that we are trying to characterize the population of asteroids in the Solar System.  Maybe their sizes have a Gaussian distribution (with some characteristic size), or maybe they have a flat distribution (with equal numbers over a large range of sizes).  Or maybe the distribution is a power-law, with lots of little asteroids and very few big ones.  Or maybe it is a power-law in the other direction: very few little ones and lots of big ones.  If you are the first person to discover asteroids, then *you don't know*.  Our job is to figure that out: based entirely on the data.

That leads us to the need for **estimators**.  Since we don't know the distribution, we have to estimate it.  

So, the book spends a lot of time talking about estimators and possible distributions.  

Let's first review some commonly computed statistical properties of a data set.

In [1]:
# Execute this cell to generate an array with 1000 random numbers drawn from a uniform distribution
import numpy as np
import scipy.stats
from astroML import stats as astroMLstats
np.random.seed(42) #So that everyone will get the same results
data = np.random.random(1000)

The **arithmetic mean** (or Expectation value) is

$$\mu = E(x) = \int_{-\infty}^{\infty} x h(x) dx,$$

where $h(x)$ must be properly normalized and the integral gets replaced by a sum for discrete distributions.

Specifically, this is the expecation value of $x$.  If you want the expectation value of something else--say $x^2$ or $(x-\mu)^2$, you replace the $x$ in the integral with that.

We could/should really think about this as the expected location (if the model is a Gaussian, where do you center your Gaussian).

In [2]:
# Execute this cell
mean = np.mean(data)
print(mean)

0.49025655332


While it is perhaps most common to compute the mean, the median is a more *robust* estimator of the (true) mean location of the distribution.  That's because it is less affected by outliers.

In [3]:
# Execute this cell.  Think about what it is doing.
median = np.median(data)
mask = data>0.75
data[mask] = data[mask]*2
newmedian = np.median(data)
newmean = np.mean(data)
print(median,newmedian)
print(mean,newmean)

(0.4968073765468109, 0.4968073765468109)
(0.49025655332013363, 0.70350427351432154)


In addition to the "average", we'd like to know something about **deviations** from the average.  The simplest thing to compute is $$d_i = x_i - \mu.$$  However, the average deviation is zero by definition of the mean.  The next simplest thing to do is to compute the mean absolute deviation (MAD):
$$\frac{1}{N}\sum|x_i-\mu|,$$
but the absolute values can hide the true scatter of the distribution [in some cases (see footnote)](http://www.mathsisfun.com/data/standard-deviation.html).  So the next simplest thing to do is to square the differences $$\sigma^2 = \frac{1}{N}\sum(x_i-\mu)^2,$$ which we call the **variance**.

Indeed the *variance* is just expectation value of $(x-\mu)^2$

$$\sigma^2 = V = \int_{-\infty}^{\infty}  (x-\mu)^2 h(x) dx,$$

where, again,  the integral gets replaced by a sum for discrete distributions.

And we define the **standard deviation** as
$$\sigma = \sqrt{V}$$

In [4]:
# Execute this cell
var = np.var(data)
std = np.std(data)
print(var,std)

(0.3956838769784139, 0.62903408252527448)


There is also the Median Absolute Deviation (also MAD) given by
$${\rm median} (|x_i-{\rm median}(\{x_i\})|)$$
where $\sigma = 1.4826\,{\rm MAD}$ for a Gaussian distribution (but note that we used a uniform distribution above).

In [5]:
from astropy.stats import median_absolute_deviation
MAD = median_absolute_deviation(data)
print(MAD,MAD*1.4826)

(0.25485359534083485, 0.37784594045232173)


Percentiles, $q_p$, are computed as
$$\frac{p}{100} = \int_{-\infty}^{q_p}h(x) dx$$

For example, the 25th, 50th, and 75th percentiles:

In [6]:
q25,q50,q75 = np.percentile(data,[25,50,75])
print(q25,q50,q75)

(0.23597326758870019, 0.4968073765468109, 0.74431958797559017)


Where we call the difference between the 25th and 75th percentiles, $q_{75} - q_{25}$, the *interquartile range*.

The median and interquartile range are more _robust_ than the mean and standard deviation.  So, one can create a standard-deviation-like measurement (at least for a Gaussian) from the interquartile range as
$\sigma_G = 0.7413(q_{75} - q_{25})$, which we saw last time.  One reason to use this is the same as for the median.  $\sigma_G$ is a more *robust* estimator of the scale of the distribution.  The normalization makes it *unbiased* for a perfect Gaussian (more on that later).

In [7]:
# Execute this cell
astroMLstats.sigmaG(data)

0.37683769118738131

The mode is the most probable value, determined from the peak of the distribution, which is the value where the derivative is 0:
$$ \left(\frac{dh(x)}{dx}\right)_{x_m} = 0$$

Another way to estimate the mode (at least for a Gaussian distribution) is
$$x_m = 3q_{50} - 2\mu$$

In [8]:
# Execute this cell  (note that data is not Gaussian so these are very different!)
mode = scipy.stats.mode(data)
modealt = 3*q50 - 2*mean
print(mode)
print(modealt)

ModeResult(mode=array([ 0.00463202]), count=array([1]))
0.509909023


Other useful measures include the "higher order" moments (the skewness and kurtosis):

$$\Sigma = \int_{-\infty}^{\infty}  \left(\frac{x-\mu}{\sigma}\right)^3 h(x) dx,$$

$$K = \int_{-\infty}^{\infty}  \left(\frac{x-\mu}{\sigma}\right)^4 h(x) dx  - 3.$$


In [9]:
# Execute this cell
skew = scipy.stats.skew(data)
kurt = scipy.stats.kurtosis(data)
print(skew,kurt)

(0.9337774639548498, -0.6644510140825068)


In [10]:
# Excute this cell
print(mean, median, var, std)
print(skew, kurt, mode.mode, modealt)
print(q25, q50, q75)

(0.49025655332013363, 0.4968073765468109, 0.3956838769784139, 0.62903408252527448)
(0.9337774639548498, -0.6644510140825068, array([ 0.00463202]), 0.5099090230001655)
(0.23597326758870019, 0.4968073765468109, 0.74431958797559017)


We could do the same with a normal distribution with a pdf given by:
 $$p(x|\mu,\sigma) = \frac{1}{\sigma\sqrt{2\pi}} \exp\left(\frac{-(x-\mu)^2}{2\sigma^2}\right).$$

In [11]:
# Complete and Execute this cell
# loc = mean (mu)
# scale = stddev (sigma)
ndata = np.random.normal(loc=0,scale=1,size=10000)

In [12]:
# Compute all the above stats for this distribution
print np.mean(ndata), np.median(ndata), np.var(ndata), np.std(ndata)
print scipy.stats.skew(ndata), scipy.stats.kurtosis(ndata), scipy.stats.mode(ndata).mode
print np.percentile(ndata, [25,50,75])

-0.00314864913533 -0.00342765299208 1.00750166094 1.00374382237
-0.0112049235769 0.0112632214933 [-3.92240025]
[-0.67203638 -0.00342765  0.6727829 ]


### Sample vs. Population Statistics 

Statistics estimated from the *data* are called _sample statistics_ as compared to _population statistics_ which come from knowing the functional form of the pdf.  Up to now we have been computing population statistics.

Specifically, $\mu$ is the *population average*, i.e., it is the expecation value of $x$ for $h(x)$.  But we don't *know* $h(x)$.  So the **sample mean**, $\overline{x}$, is an *estimator* of $\mu$, defined as
$$\overline{x} \equiv \frac{1}{N}\sum_{i=1}^N x_i,$$
which we determine from the data itself.

Then instead of $\sigma^2$, which is the population variance, we have the **sample variance**, $s^2$, where

$$s^2 = \frac{1}{N-1}\sum_{i=1}^N(x_i-\overline{x})^2$$

Where it is $N-1$ instead of $N$ since we had to determine $\overline{x}$ from the data instead of using a known $\mu$.  Ideally one tries to work in a regime where $N$ is large enough that we can be lazy and ignore this. 

So the mean and variance of a distribution are $\mu$ and $\sigma^2$.  The *estimators* of the distribution are $\overline{x}$ (or $\hat{x}$) and $s^2$.

### Bias

If there is a difference between the *estimator* and the *population* values, we say that the estimator is **biased** (perhaps not quite the usage of the word that you are used to).  E.g., if your distribution is Gaussian and $\overline{x}$ is a biased estimator of $\mu$, then the Gaussian model is not centered in the right place.

Again, more on this later.

### Uncertainty

We would also like to know the uncertainty of our estimates $\overline{x}$ and $s$.  Note that $s$ is **NOT** the uncertainty of $\overline{x}$.  Rather the uncertainty of $\overline{x}$, $\sigma_{\overline{x}}$ is 
$$ \sigma_{\overline{x}} = \frac{s}{\sqrt{N}},$$
which we call the *standard error of the mean*.  So, the accuracy to which we know the mean is smaller than the width of the distribution.

The uncertainty of $s$ itself is
$$\sigma_s = \frac{s}{\sqrt{2(N-1)}} = \frac{1}{\sqrt{2}}\sqrt{\frac{N}{N-1}}\sigma_{\overline{x}}.$$

Note that for large $N$, $\sigma_{\overline{x}} \sim \sqrt{2}\sigma_s$ and for small $N$, $\sigma_s$ is not much smaller than $s$.